<a href="https://colab.research.google.com/github/NewTech-bit/DLB/blob/AK_1/DLB_FinalDataChallenge_XL_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is the final data challenge project for Deep Learning Datacamp at HSLU in September 2021. It's a group project conducted by Andreas Kläusli, Kevin Bollier, Ricky Roy Bruderer, and Xiying Liu.

## Import

In [115]:
import os
import os.path
import pathlib
import numpy as np
import pandas as pd
import cv2
import random
import shutil
import re

import PIL
import PIL.Image
import IPython.display as display

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.models import Model
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop, Adam

from tensorflow import keras
from tensorflow.keras import layers

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline


In [ ]:
# print(len([name for name in os.listdir('.') if os.path.isfile(name)]))

0


In [3]:
os.listdir()

['.config', 'sample_data']

## Load data

Data is uploaded by accessing files in Google Drive.

In [4]:
# mount google drive to colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
train_path = '/content/drive/MyDrive/DLB/training_data'
test_path = '/content/drive/MyDrive/DLB/validation_data'
train_dir = pathlib.Path(train_path)
test_dir = pathlib.Path(test_path)

In [ ]:
train_dir

PosixPath('/content/drive/MyDrive/DLB/training_data')

In [ ]:
# builder = tfds.ImageFolder(test_path)
# print(builder.info)
# ds = builder.as_dataset(shuffle_files=True)
# # tfds.show_examples(ds, builder.info)
# tfds.show_examples(ds, ds_info)

In [6]:
train_image_count = len(list(train_dir.glob('*.jpg')))
test_image_count = len(list(test_dir.glob('*.jpg')))
print("Number of images in training dataset: ", train_image_count)
print("Number of images in testing dataset: ", test_image_count)

Number of images in training dataset:  0
Number of images in testing dataset:  985


In [7]:
cats_train = list(train_dir.glob('cat.*.jpg'))
dogs_train = list(train_dir.glob('dog.*.jpg'))
cats_test = list(test_dir.glob('cat.*.jpg'))
dogs_test = list(test_dir.glob('dog.*.jpg'))


cats_train_count = len(cats_train)
dogs_train_count = len(dogs_train)
cats_test_count = len(cats_test)
dogs_test_count = len(dogs_test)
print("Number of cat images in training: ", cats_train_count)
print("Number of dog images in training: ", dogs_train_count)
print('\n')
print("Number of cat images in testing: ", cats_test_count)
print("Number of dog images in testing: ", dogs_test_count)

Number of cat images in training:  0
Number of dog images in training:  0


Number of cat images in testing:  496
Number of dog images in testing:  489


In [8]:
# # define a function to count images
# def count_image(file_dir, image_class):
#   image_count = len(list(file_dir.glob('*.jpg')))
#   image_class_count = len(list(file_dir.glob('image_class.*.jpg')))
#   return image_count, image_class_count

In [10]:
for image_path in list(train_dir.glob('*.jpg'))[:3]:
  display.display(PIL.Image.open(str(image_path)))

### Preprocessing for ingestion in keras.preprocessing flow
https://vijayabhaskar96.medium.com/tutorial-image-classification-with-keras-flow-from-directory-and-generators-95f75ebe5720

In [130]:
# restructure train folder into subfolders cat and dog; move labelled files into
# subfolders
def create_subfolders(root_path, labels):
  """
  creates subfolders for each category within the root_path folder
  :param root_path: string path with dataset in it
  :param labels: list of category labels
  """
  root = pathlib.Path(root_path)
  path_list = []
  for label in labels:
    train_label_path = root_path+"/"+label
    path_list.append(train_label_path)
    try:
      pathlib.Path(train_label_path).mkdir(parents=True, exist_ok=False)
    except FileExistsError:
      print("Folder {} is already there".format(train_label_path))
    else:
      print("Folder {} was created".format(train_label_path))
    print(path_list)
  return path_list

def files_to_subfolders(root_path, labels, path_list):
  """
  moves files into the subfolders based on filenames 'cat.XXX.jpg'
  :root_path: path within datasets are located
  :param labels: list of category labels
  :param path_list: list of subfolder paths
  """
  root = pathlib.Path(root_path)
  cont_var = 0
  for folder in path_list:
    path = pathlib.Path(folder)
    if len(list(path.glob('*.*'))) == 0:
      print("Folder {} is empty, process is ongoing".format(folder))
    else:
      print("Folder {} is not empty, check content of the folder".format(folder))
      cont_var = 1
  if cont_var == 0:
    for file in root.glob('*.*.jpg'):
      folder_name = file.stem.split(".", 1)[0]
      file.rename(root /folder_name / file.name)
      
  else:
    print("Files could not have been moved to training subfolders")


In [121]:
root_path=train_path
labels=['cat','dog']
t = create_subfolders(root_path=root_path, labels = labels)
files_to_subfolders(root_path=root_path, labels = labels, path_list=t)

Folder /content/drive/MyDrive/DLB/training_data/cat is already there
['/content/drive/MyDrive/DLB/training_data/cat']
Folder /content/drive/MyDrive/DLB/training_data/dog is already there
['/content/drive/MyDrive/DLB/training_data/cat', '/content/drive/MyDrive/DLB/training_data/dog']
Folder /content/drive/MyDrive/DLB/training_data/cat is not empty, check content of the folder
Folder /content/drive/MyDrive/DLB/training_data/dog is not empty, check content of the folder
Files could not been moved to subfolder


In [127]:
# generate validation for training (10 %) per category
# create validation folder DLB/train_val 

def create_train_val(dest_path, frac, path_list, labels, seed):
  """
  creates folder for training validation with fraction of files per category 
  within
  :param dest_path: path of training validation datasets
  :param frac: random fraction per category
  :param path_list: list of paths containing datasets per category
  :param seed:
  """
# create folder for training validation if not exists
  try:
    pathlib.Path(dest_path).mkdir(parents=True, exist_ok=False)
  except FileExistsError:
    print("Folder {} already exists".format(dest_path))
  else:
    print("Folder {} created".format(dest_path))
# create subfolders per category
  l_path = create_subfolders(dest_path, labels)

# move fraction of randomised files into subfolders
  random.seed(seed)
  for path in path_list:
    filenames = random.sample(os.listdir(path), int(frac*len(list(pathlib.Path(path).glob('*.*')))))
    print("Number of randomised files from {0}: {1}".format(path, len(filenames)))
    for fname in filenames:
      srcpath = os.path.join(path, fname)
      reg =re.findall(r'^\w+', fname)[0]
      dst_path = dest_path+"/"+reg
      despath = os.path.join(dst_path, fname)
      shutil.copyfile(str(srcpath), str(despath))
      del dst_path
  print("train_val generated.")

In [129]:
create_train_val(dest_path="/content/drive/MyDrive/DLB/train_val", frac=0.1, seed=5, path_list = t, labels = labels)

Folder /content/drive/MyDrive/DLB/train_val created
Folder /content/drive/MyDrive/DLB/train_val/cat was created
['/content/drive/MyDrive/DLB/train_val/cat']
Folder /content/drive/MyDrive/DLB/train_val/dog was created
['/content/drive/MyDrive/DLB/train_val/cat', '/content/drive/MyDrive/DLB/train_val/dog']
Number of randomised files from /content/drive/MyDrive/DLB/training_data/cat: 150
Number of randomised files from /content/drive/MyDrive/DLB/training_data/dog: 140
train_val generated.


## Load data

Using `tf.data.Dataset`

Define some parameters for the loader:

In [ ]:
batch_size = 17
img_height = 200
img_width = 200

In [ ]:
# show random five images from training dataset
plt.figure(figsize=(20,20))

for i in range(5):
  file = random.choice(os.listdir(train_path))
  image_path = os.path.join(train_path, file)
  img = mpimg.imread(image_path)
  ax = plt.subplot(1, 5, i+1)
  ax.title.set_text(file)
  plt.imshow(img)

FileNotFoundError: ignored

<Figure size 1440x1440 with 0 Axes>

In [ ]:
os.listdir(test_path)[3] # check naming convention of images

'dog.4713.jpg'

In [ ]:
# setting parameters for loading the dataset
IMG_WIDTH = 128
IMG_HEIGHT = 128
# train_path = '/content/drive/MyDrive/DLB/training_data'
# test_path = '/content/drive/MyDrive/DLB/validation_data'

In [ ]:
# creating the image data and the labels

def create_dataset(img_folder):
  img_data_array=[]
  class_name=[]

  for file in os.listdir(img_folder):
    image_path = os.path.join(img_folder, file)
    cls_name = file[:3]
    image = cv2.imread(image_path, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH), interpolation=cv2.INTER_AREA)
    image = np.array(image)
    image = image.astype('float32')
    image = image / 255.0
    img_data_array.append(image)
    class_name.append(cls_name)
  
  return img_data_array, class_name

In [ ]:
# create test dataset with (image, lable) pairs, takes ~6 mins
test_image, test_label = create_dataset(test_path)

In [ ]:
# create train dataset with (image, lable) pairs, takes ~17 mins
train_image, train_label = create_dataset(train_path)

In [ ]:
def class_encode(label):
  target_dict = {k:v for v, k in enumerate(np.unique(label))}
  target_value = [target_dict[label[i]] for i in range(len(label))]
  return target_value

In [ ]:
train_label = class_encode(train_label)
test_label = class_encode(test_label)

NameError: ignored

## Create deep learning models

In [ ]:
input_shape=(64,64,3)

In [ ]:
# # build a convolutional neural network withouth image argumentation
# def func_model():
#   input_layer = tf.keras.Input(shape=input_shape) # instantiate the input layer

#   x = Conv2D(16, (3,3), activation='relu')(input_layer) # stack layers using the functional API syntax
#   x = MaxPooling2D(2,2)
#   x = Conv2D(32, (3,3), activation='relu')(x)
#   x = MaxPooling2D(2,2)
#   x = Flatten()(x)
#   x = Dense(128, activation='relu')(x)

#   output_layer = Dense(1, activation='sigmoid')(x)

#   func_model = Model(inputs = input_layer, outputs=output_layer) # declare inputs and outputs

### 1. Baseline CNN model

In [ ]:
def model_conv():
  model = Sequential([Conv2D(32, (3,3), activation='relu', input_shape=(64,64,3)),
                      MaxPooling2D(2,2),
                      Conv2D(64, (3,3), activation='relu'),
                      MaxPooling2D(2,2),
                      Flatten(),
                      Dense(128, activation='relu'),
                      Dense(1, activation='sigmoid')  
  ])

In [ ]:
model = func_model()
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(learning_rate=1e-4),
              metrics=['accuracy'])

ValueError: ignored

In [ ]:
# flow images in batches of 20 using _datagen generator
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

image_size = (150, 150)
train_generator = train_datagen.flow_from_directory(train_path,
                                                    target_size=image_size,
                                                    batch_size=20,
                                                    class_mode='binary')
test_generator = test_datagen.flow_from_directory(test_path,
                                                   target_size=image_size,
                                                   batch_size=20,
                                                   class_mode='binary')

Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=171, # 2907 images = batch_size 17 * steps 171
    epochs = 15,
    validation_data= validation_generator,
    validation_steps=58,
    verbose=2
)

In [ ]:
# plot results
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label="Training accuracy")
plt.plot(epochs, val_acc, 'b', label="Validation accuracy")
plt.title("Training and validation accuracy")

plt.figure()

plt.plot(epochs, loss, 'bo', label="Training loss")
plt.plot(epochs, val_loss, 'b', label="Validation loss")
plt.title("Training and validation loss")
plt.legen()

plt.show()

### CNN model with image augmentation
-> to be completed: data_augmentation with preprocessing flow

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
    ]
)

In [ ]:
inputs = keras.Input(shape=input_shape)
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(x)

In [ ]:
def model_conv2():
  model = Sequential([Conv2D(32, (3,3), activation='relu', input_shape=(64,64,3)),
                      MaxPooling2D(2,2),
                      Conv2D(64, (3,3), activation='relu'),
                      MaxPooling2D(2,2),
                      Flatten(),
                      Dense(128, activation='relu'),
                      Dense(1, activation='sigmoid')
 
  ])


In [ ]:
inputs = keras.Input(shape=input_shape)
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(x)

### Transfer learning

## Prediction